## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-01-05-39 +0000,bbc,White House begins demolishing part of East Wi...,https://www.bbc.com/news/articles/ced6np51532o...
1,2025-10-21-01-00-56 +0000,latimes,Newsom warns Californians' SNAP benefits could...,https://www.latimes.com/california/story/2025-...
2,2025-10-21-00-58-45 +0000,nyt,"With SpaceX Behind Schedule, NASA Will Seek Mo...",https://www.nytimes.com/2025/10/20/science/nas...
3,2025-10-21-00-58-44 +0000,48hills,Screen Grabs: East LA guerrilla artists fire u...,https://48hills.org/2025/10/screen-grabs-east-...
4,2025-10-21-00-53-48 +0000,nypost,Curtis Sliwa’s checkered past catches up to hi...,https://nypost.com/2025/10/20/us-news/curtis-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2442/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,38
58,louvre,16
263,new,15
155,china,13
67,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
101,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...,93
32,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...,89
22,2025-10-20-23-45-15 +0000,nyt,Appeals Court Ruling Allows Trump to Deploy Na...,https://www.nytimes.com/2025/10/20/us/appeals-...,85
112,2025-10-20-19-28-34 +0000,nypost,Demolition begins on East Wing of White House ...,https://nypost.com/2025/10/20/us-news/demoliti...,84
81,2025-10-20-21-06-07 +0000,bbc,Appeals court allows Trump to deploy National ...,https://www.bbc.com/news/articles/c201k3qky9lo...,79


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
101,93,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...
32,51,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...
112,46,2025-10-20-19-28-34 +0000,nypost,Demolition begins on East Wing of White House ...,https://nypost.com/2025/10/20/us-news/demoliti...
186,43,2025-10-20-12-48-54 +0000,bbc,Deadly Gaza flare-up tests Israel-Hamas ceasefire,https://www.bbc.com/news/articles/czxk8k4xlv1o...
162,42,2025-10-20-15-10-02 +0000,nypost,French authorities admit they ‘failed’ in Louv...,https://nypost.com/2025/10/20/world-news/louvr...
20,40,2025-10-20-23-48-58 +0000,bbc,US and Australia sign rare earths deal to coun...,https://www.bbc.com/news/articles/cly9kvrdk2xo...
246,35,2025-10-20-05-21-50 +0000,nyt,Sanae Takaichi Is Set to Be Japan’s First Fema...,https://www.nytimes.com/2025/10/19/world/asia/...
176,32,2025-10-20-13-30-46 +0000,latimes,"Why did Amazon's AWS outage disrupt Snapchat, ...",https://www.latimes.com/business/story/2025-10...
253,32,2025-10-20-02-37-11 +0000,nypost,Thousands of Orthodox Jews protest around Isra...,https://nypost.com/2025/10/19/us-news/thousand...
75,30,2025-10-20-21-18-51 +0000,bbc,Former NFL player Doug Martin dies in police c...,https://www.bbc.com/news/articles/c6297qm94z0o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
